In [1]:
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation
from keras.layers import LSTM
import matplotlib.pyplot as plt
import matplotlib
import time
import pandas as pd
import numpy as np
%matplotlib qt
from numpy.random import choice
from keras.callbacks import Callback

Using TensorFlow backend.


In [16]:
N_train = 1000
X_train = np.zeros((1200,20))


one_indexes = choice(a=N_train, size=N_train / 2, replace=False)
X_train[one_indexes, 0] = 1  # very long term memory.
Y_train = X_train[:,0]
x_tr,y_tr = X_train[:1000,],Y_train[:1000,]
x_ts,y_ts = X_train[-200:,],Y_train[-200:,]


def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)
X_train, y_train = prepare_sequences(x_tr,y_tr,20)
X_test, y_test = prepare_sequences(x_ts,y_ts,20)

# reshape
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)
max_len = 20
batch_size = 1
print('sequences_x_train shape:', X_train.shape)
print('sequences_y_train shape:', y_train.shape)

print('sequences_x_test shape:', X_test.shape)
print('sequences_y_test shape:', y_test.shape)

('sequences_x_train shape:', (1000, 20, 1))
('sequences_y_train shape:', (1000, 1))
('sequences_x_test shape:', (200, 20, 1))
('sequences_y_test shape:', (200, 1))


In [17]:
X_train[0]

array([[ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

Impact of subsequence subsampling

In [12]:
USE_SEQUENCES = False
max_len = 20
batch_size = 1

N_train = 1000
N_test = 200

X_train = np.zeros((N_train, max_len))
X_test = np.zeros((N_test, max_len))

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

y_train = np.zeros((N_train, 1))
y_test = np.zeros((N_test, 1))

one_indexes = choice(a=N_train, size=N_train / 2, replace=False)
X_train[one_indexes, 0] = 1
y_train[one_indexes] = 1

one_indexes = choice(a=N_test, size=N_test / 2, replace=False)
X_test[one_indexes, 0] = 1
y_test[one_indexes] = 1

if USE_SEQUENCES:
    max_len = 10
    X_train, y_train = prepare_sequences(X_train, y_train, window_length=max_len)
    X_test, y_test = prepare_sequences(X_test, y_test, window_length=max_len)

X_train = np.expand_dims(X_train, axis=2)  # input dim is 1. Timesteps is the sequence length.
X_test = np.expand_dims(X_test, axis=2)

print('sequences_x_train shape:', X_train.shape)
print('sequences_y_train shape:', y_train.shape)

print('sequences_x_test shape:', X_test.shape)
print('sequences_y_test shape:', y_test.shape)


('X_train shape:', (1000, 20))
('X_test shape:', (200, 20))
('sequences_x_train shape:', (1000, 20, 1))
('sequences_y_train shape:', (1000, 1))
('sequences_x_test shape:', (200, 20, 1))
('sequences_y_test shape:', (200, 1))


In [13]:
X_train[0]

array([[ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [3]:
class ResetStatesCallback(Callback):
    def __init__(self):
        self.counter = 0

    def on_batch_begin(self, batch, logs={}):
        if self.counter % max_len == 0:
            self.model.reset_states()
        self.counter += 1

In [18]:
print('Build STATELESS model...')
model = Sequential()
model.add(LSTM(10, input_shape=(max_len, 1), return_sequences=False, stateful=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Train...')
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=15,
              validation_data=(X_test, y_test), shuffle=False, callbacks=[ResetStatesCallback()])

plt.figure()
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.title('model loss stateless')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()   

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
print('___________________________________')
print('Test score:', score)
print('Test accuracy:', acc)

Build STATELESS model...
Train...


/home/cloud/.virtualenvs/deeplearn/local/lib/python2.7/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1000 samples, validate on 200 samples
Epoch 1/15
1000/1000 [==============================] - 13s - loss: 0.1315 - acc: 0.9540 - val_loss: 0.0098 - val_acc: 1.0000
Epoch 2/15
1000/1000 [==============================] - 13s - loss: 0.0038 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 3/15
1000/1000 [==============================] - 13s - loss: 0.0012 - acc: 1.0000 - val_loss: 8.6233e-04 - val_acc: 1.0000
Epoch 4/15
1000/1000 [==============================] - 12s - loss: 5.6840e-04 - acc: 1.0000 - val_loss: 4.6509e-04 - val_acc: 1.0000
Epoch 5/15
1000/1000 [==============================] - 12s - loss: 3.0010e-04 - acc: 1.0000 - val_loss: 2.5785e-04 - val_acc: 1.0000
Epoch 6/15
1000/1000 [==============================] - 12s - loss: 1.6582e-04 - acc: 1.0000 - val_loss: 1.4590e-04 - val_acc: 1.0000
Epoch 7/15
1000/1000 [==============================] - 12s - loss: 9.3940e-05 - acc: 1.0000 - val_loss: 8.3512e-05 - val_acc: 1.0000
Epoch 8/15
1000/1000 [=============

In [ ]:
Y_train.shape

In [4]:
# STATEFUL MODEL
print('Build STATEFUL model...')
model = Sequential()
model.add(LSTM(10,
               batch_input_shape=(1, 1, 1), return_sequences=False,
               stateful=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

x = np.expand_dims(np.expand_dims(X_train.flatten(), axis=1), axis=1)
y = np.expand_dims(np.array([[v] * max_len for v in y_train.flatten()]).flatten(), axis=1)
model.fit(x,
          y,
          callbacks=[ResetStatesCallback()],
          batch_size=1,
          shuffle=False)

print('Train...')
for epoch in range(15):
    mean_tr_acc = []
    mean_tr_loss = []
    for i in range(len(X_train)):
        y_true = y_train[i]
        for j in range(max_len):
            tr_loss, tr_acc = model.train_on_batch(np.expand_dims(np.expand_dims(X_train[i][j], axis=1), axis=1),
                                                   np.array([y_true]))
            mean_tr_acc.append(tr_acc)
            mean_tr_loss.append(tr_loss)
        model.reset_states()

    print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
    print('loss training = {}'.format(np.mean(mean_tr_loss)))
    print('___________________________________')

    mean_te_acc = []
    mean_te_loss = []
    for i in range(len(X_test)):
        for j in range(max_len):
            te_loss, te_acc = model.test_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1),
                                                  y_test[i])
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
        model.reset_states()

        for j in range(max_len):
            y_pred = model.predict_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1))
        model.reset_states()

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
    print('___________________________________')

Build STATEFUL model...
Epoch 1/10
20000/20000 [==============================] - 43s - loss: 0.0459 - acc: 0.9758    
Epoch 2/10
20000/20000 [==============================] - 45s - loss: 1.7573e-07 - acc: 1.0000    
Epoch 3/10
20000/20000 [==============================] - 47s - loss: 1.0960e-07 - acc: 1.0000    
Epoch 4/10
20000/20000 [==============================] - 46s - loss: 1.0960e-07 - acc: 1.0000    
Epoch 5/10
20000/20000 [==============================] - 44s - loss: 1.0960e-07 - acc: 1.0000    
Epoch 6/10
20000/20000 [==============================] - 43s - loss: 1.0960e-07 - acc: 1.0000    
Epoch 7/10
20000/20000 [==============================] - 44s - loss: 1.0960e-07 - acc: 1.0000    
Epoch 8/10
20000/20000 [==============================] - 42s - loss: 1.0960e-07 - acc: 1.0000    
Epoch 9/10
20000/20000 [==============================] - 43s - loss: 1.0960e-07 - acc: 1.0000    
Epoch 10/10
20000/20000 [==============================] - 44s - loss: 1.0960e-07 - acc: 